In [1]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score

In [2]:
import os
os.getcwd()

'/Users/sd363/Desktop/crowdfunding'

In [3]:
df = pd.read_csv('Super earners/qualtrics/responses/1_28_2021_ada_anu.csv')

In [4]:
df.loc[:,'qid'] = [x.split('-')[0] for x in df['question_id']]

In [5]:
df['qid'].unique()

array(['individual_or_organization', 'links_external', 'appeals',
       'author_relationship', 'patient_culpability', 'age', 'gender',
       'family_role', 'career', 'career_notes',
       'positive_char_ref_patient', 'positive_char_ref_other',
       'insurance', 'mention_funds', 'non_medical_funds', 'medical_funds',
       'other_funds', 'notes'], dtype=object)

In [83]:
def Agreement(l1,l2):
    correct = 0
    for i in range(len(l1)):
        if l1[i] == l2[i]:
            correct = correct + 1
        else:
            continue
    return correct/len(l1)

In [7]:
r = []
qs = [
    'individual_or_organization',
    'links_external',
    'appeals',
    'author_relationship',
    'patient_culpability',
    'age',
    'gender',
    'career',
    'positive_char_ref_patient',
    'positive_char_ref_other',
    'insurance',
    'mention_funds',
    'other_funds'
    
]
for q in qs:
    d = df[df['qid'] == q]
    r1 = d['rater_1'].tolist()
    r2 = d['rater_2'].tolist()
    ck = cohen_kappa_score(r1,r2)
    agreement = Agreement(r1,r2)
    r.append({'qid':q, 'ck': ck, 'agr': agreement})

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:604: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


In [8]:
pd.DataFrame(r)

,qid,ck,agr
0,individual_or_organization,NaN,1.00
1,links_external,0.827586,0.95
2,appeals,0.875000,0.95
3,author_relationship,0.861111,0.90
4,patient_culpability,NaN,1.00
5,age,0.701493,0.90
6,gender,1.000000,1.00
7,career,0.615385,0.80
8,positive_char_ref_patient,0.693487,0.80
9,positive_char_ref_other,0.733333,0.90


In [100]:
med_funds = df[df['qid'] == 'non_medical_funds'].fillna(' ')

In [99]:
med_funds.head()

,question_id,question_text,rater_1,rater_2,qid
14,non_medical_funds,Does the campaign describe intended use of fun...,Accounting for lost earnings,"Travel,Other non-medical expense (not otherwis...",non_medical_funds
32,non_medical_funds-1,Does the campaign describe intended use of fun...,NaN,Does not mention non-medical expenses or finan...,non_medical_funds
50,non_medical_funds-2,Does the campaign describe intended use of fun...,Does not mention non-medical expenses or finan...,Does not mention non-medical expenses or finan...,non_medical_funds
68,non_medical_funds-3,Does the campaign describe intended use of fun...,Does not mention non-medical expenses or finan...,Does not mention non-medical expenses or finan...,non_medical_funds
86,non_medical_funds-4,Does the campaign describe intended use of fun...,"Travel,Other non-medical expense (not otherwis...",Does not mention non-medical expenses or finan...,non_medical_funds


In [101]:
med_funds.loc[:,'rater_1_clean'] = [x.split(',') for x in med_funds['rater_1']]

In [102]:
med_funds.loc[:,'rater_2_clean'] = [x.split(',') for x in med_funds['rater_2']]

In [103]:
#convert data from wide to long
rater_1 = pd.melt(med_funds, id_vars=['question_id'], value_vars=['rater_1_clean']).explode('value')
rater_2 = pd.melt(med_funds, id_vars=['question_id'], value_vars=['rater_2_clean']).explode('value')

In [104]:
melt = pd.concat([rater_1, rater_2])

In [105]:
melt = melt.rename(columns={'variable':'rater','value':'cat'})
melt = melt.reset_index()
del melt['index']

In [106]:
def comma2lofd(df):
    '''convert cat column from comma delimited to list of dictionaries with 1 or 0 for each category'''
    
#     disease_categories = [
#         'Medical expenses - partly covered by insurance',
#         'Medial expenses - not covered by insurance',
#         'Medical expenses - no mention of insurance coverage',
#         'Does not mention medical expeneses or financial burden'
#     ]

    disease_categories = [
        'Accounting for lost earnings',
        'Travel',
        'Does not mention non-medical expenses or financial burden',
        'Other non-medical expense (not otherwise specified)',
        'Child care'
    ]
    
    r = []
    
    for index,row in df.iterrows():
        lofd = []
        as_list = row['cat'].split(',')
        for disease in disease_categories:
            if disease in as_list:
                lofd.append({disease:1})
            else:
                lofd.append({disease:0})
        r.append(lofd)
    return r

In [107]:
melt['lofd'] = comma2lofd(melt)
del melt['cat']

In [108]:
long = melt.explode('lofd')

In [109]:
long.insert(loc=2, column='cat', value=[list(x.items())[0][0] for x in long['lofd']])
long.insert(loc=3, column='count', value=[list(x.items())[0][1] for x in long['lofd']])

In [110]:
rater_1_pivot = long[long['rater'] == 'rater_1_clean'].pivot_table(index=['question_id'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)

In [111]:
rater_2_pivot = long[long['rater'] == 'rater_2_clean'].pivot_table(index=['question_id'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)

In [112]:
cat = []
cohens_kappa = []
percent_agreement = []
for col in rater_1_pivot.columns:
    cat.append(col)
    cohens_kappa.append(cohen_kappa_score(rater_1_pivot[col], rater_2_pivot[col]))
    percent_agreement.append(Agreement(rater_1_pivot[col], rater_2_pivot[col]))
kappa_df = pd.DataFrame({'category':cat, 
              'cohens kappa':cohens_kappa,
              'agr':percent_agreement})

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:604: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


In [113]:
kappa_df

,category,cohens kappa,agr
0,Accounting for lost earnings,0.230769,0.8
1,Child care,NaN,1.0
2,Does not mention non-medical expenses or finan...,0.200000,0.6
3,Other non-medical expense (not otherwise speci...,0.529412,0.8
4,Travel,0.230769,0.8


In [7]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from time import sleep
import requests 
import re
import json



page = requests.get("https://www.gofundme.com/f/eric-stevens-care-trust")

soup = BeautifulSoup(page.content, 'html.parser')

all_scripts = soup.find_all('script')
txt = all_scripts[0].get_text()
data = json.loads(re.findall(r'window\.initialState = ({.*?});', txt)[0])

In [8]:
data

{'auth': {'captchaRequired': False,
  'captchaValid': False,
  'captchaValidating': False,
  'email': {'error': {}, 'status': 'default'},
  'facebook': {'error': {}, 'status': 'default'},
  'isAuthenticated': False},
 'bigweb': {'contact': {'actions': [],
   'chatAudience': 1,
   'chatReady': False,
   'chatStatus': 'offline',
   'chatWindow': 'closed',
   'emailAudience': 0,
   'isLoadingActions': False,
   'selected': [0],
   'suggestions': {}}},
 'co': {'createComment': {'media': []},
  'donationReceipt': {'amount': 0,
   'currencyCode': '',
   'donor': {'email': '', 'firstName': '', 'isAnonymous': True, 'name': ''},
   'signedDonationId': ''},
  'postDonateCreateComment': {'error': False,
   'loading': False,
   'media': [],
   'mediaError': []},
  'teamEmailInvites': {'error': False,
   'field_errors': {},
   'invitation_count': 0,
   'loading': False},
  'teamInvitations': {}},
 'facebook': {'email': '',
  'facebookId': '',
  'facebookToken': '',
  'first_name': '',
  'last_name'